# Recursive

In [1]:
from operator import add
def fib(n):
    if n <= 2:
        return n
    else:
        return add(fib(n-1),fib(n-2))
fib(10)

89

# Procedural

In [2]:
def end_cc(n):
    print('end of continuation')
    return n

def fib2_cc(cc,w):
    return lambda v: cc(add(w,v))

def fib1_cc(cc,n):
    return lambda w: fib_k(n-2,fib2_cc(cc,w))

def fib_k(n,cc=end_cc):
    if n <= 2:
        return cc(n)
    else:
        return fib_k(n-1,fib1_cc(cc,n))

fib_k(10)

end of continuation


89

# Data Structure

In [3]:
from dataclasses import dataclass
from typing import Callable

@dataclass
class Cont:
    cc:Callable[[int],int]

@dataclass
class End_cc(Cont):
    cc:Callable[[int],int] = None

@dataclass
class Fib1_cc(Cont):
    n:int
    
@dataclass
class Fib2_cc(Cont):
    v1:int

def fib_cc(cc,v):
    return Fib1_cc(cc,v)

def apply_cc(cc:Cont,v:int):
    if isinstance(cc,End_cc):
        print('end of continuation')
        return v
    elif isinstance(cc,Fib2_cc):
        return apply_cc(cc.cc,add(cc.v1,v))
    elif isinstance(cc,Fib1_cc):
        return fib_k(cc.n-2,Fib2_cc(cc.cc,v))
    else:
        print(cc)
        raise NotImplementedError

def fib_k(n,cc=End_cc()):
    if n <= 2:
        return apply_cc(cc,n)
    else:
        return fib_k(n-1,fib_cc(cc,n))

fib_k(10)

end of continuation


89

# Inline

In [4]:
def fib_k(n,cc=end_cc):
    if n <= 2:
        return cc(n)
    else:
        return fib_k(n-1,lambda val_1: fib_k(n-2,lambda val_2: cc(add(val_1,val_2))))
fib_k(10)

end of continuation


89

# Registerized

In [5]:
@dataclass
class Registers:
    v:None = None
    n:None = None
    cc:None = None

REG = Registers()

def fib(n):
    REG.cc = End_cc()
    REG.n = n
    fib_k()

def apply_cc():
    cc = REG.cc
    if isinstance(cc,End_cc):
        print('end of continuation')
        REG.cc = None
    elif isinstance(cc,Fib2_cc):
        REG.v = add(REG.v,cc.v1)
        REG.cc = cc.cc
        apply_cc()
    elif isinstance(cc,Fib1_cc):
        REG.cc = Fib2_cc(cc.cc,REG.v)
        REG.n = cc.n-2
        fib_k()
    else:
        print(cc)
        raise NotImplementedError

def fib_cc(cc,n):
    return Fib1_cc(cc,n)

def fib_k():
    n = REG.n
    cc = REG.cc
    if n <= 2:
        REG.v = n
        apply_cc()
    else:
        REG.cc = fib_cc(cc,n)
        REG.n = n-1
        fib_k()

fib(10)
print(REG)

end of continuation
Registers(v=89, n=2, cc=None)
